<b>This is for getting similar songs given a song from the user</b>

In [9]:
import base64
import hashlib
import hmac
import os
import time
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


## ACR cloud

In [4]:
# ACRCloud API credentials
access_key = ""
access_secret = ""
requrl = "https://identify-eu-west-1.acrcloud.com/v1/identify" 

# Request details
http_method = "POST"
http_uri = "/v1/identify"
data_type = "audio"  # For audio recognition
signature_version = "1"
timestamp = time.time()

# Create string to sign
string_to_sign = "\n".join([
    http_method,
    http_uri,
    access_key,
    data_type,
    signature_version,
    str(timestamp)
])

# Generate signature
sign = base64.b64encode(hmac.new(access_secret.encode('ascii'), string_to_sign.encode('ascii'), hashlib.sha1).digest()).decode('ascii')


In [5]:

# File to be recognized
file_path = 'DATASET/Test.mp4'
sample_bytes = os.path.getsize(file_path)

# Prepare files and data for POST request
files = {
    'sample': open(file_path, 'rb')
}
data = {
    'access_key': access_key,
    'sample_bytes': sample_bytes,
    'timestamp': str(timestamp),
    'signature': sign,
    'data_type': data_type,
    'signature_version': signature_version
}

# Send request
response = requests.post(requrl, files=files, data=data)
response.encoding = "utf-8"
print(response.text)


{"cost_time":2.0600001811981,"status":{"code":0,"msg":"Success","version":"1.0"},"result_type":0,"metadata":{"music":[{"db_begin_time_offset_ms":30160,"external_ids":{"isrc":"GBAYE1500979","upc":"825646487059"},"sample_begin_time_offset_ms":0,"sample_end_time_offset_ms":11680,"play_offset_ms":42080,"genres":[{"name":"Alternative"}],"duration_ms":249146,"db_end_time_offset_ms":41840,"label":"Parlophone UK","release_date":"2019-04-12","album":{"name":"BRIT Awards 2017"},"title":"Hymn For the Weekend","result_from":3,"acrid":"d8df8190edd31cd232f384f93263b046","artists":[{"name":"Coldplay"}],"external_metadata":{"deezer":{"artists":[{"id":892,"name":"Coldplay"}],"track":{"id":"114811546","name":"Hymn For The Weekend"},"album":{"name":"A Head Full Of Dreams","id":11898198}},"spotify":{"artists":[{"name":"Coldplay","id":"4gzpq5DPGxSnKTe4SA8HAU"}],"track":{"id":"1pqT8dY3ti8vrjdxRnhi9g","name":"Hymn For The Weekend"},"album":{"name":"NOW That's What I Call Music!, Vol. 60 (Deluxe Edition)","id

In [7]:
recognition_result = response.text
recognition_result

'{"cost_time":2.0600001811981,"status":{"code":0,"msg":"Success","version":"1.0"},"result_type":0,"metadata":{"music":[{"db_begin_time_offset_ms":30160,"external_ids":{"isrc":"GBAYE1500979","upc":"825646487059"},"sample_begin_time_offset_ms":0,"sample_end_time_offset_ms":11680,"play_offset_ms":42080,"genres":[{"name":"Alternative"}],"duration_ms":249146,"db_end_time_offset_ms":41840,"label":"Parlophone UK","release_date":"2019-04-12","album":{"name":"BRIT Awards 2017"},"title":"Hymn For the Weekend","result_from":3,"acrid":"d8df8190edd31cd232f384f93263b046","artists":[{"name":"Coldplay"}],"external_metadata":{"deezer":{"artists":[{"id":892,"name":"Coldplay"}],"track":{"id":"114811546","name":"Hymn For The Weekend"},"album":{"name":"A Head Full Of Dreams","id":11898198}},"spotify":{"artists":[{"name":"Coldplay","id":"4gzpq5DPGxSnKTe4SA8HAU"}],"track":{"id":"1pqT8dY3ti8vrjdxRnhi9g","name":"Hymn For The Weekend"},"album":{"name":"NOW That\'s What I Call Music!, Vol. 60 (Deluxe Edition)","

In [8]:
import json

result_json = json.loads(recognition_result)
spotify_track_id = result_json.get('metadata', {}).get('music', [])[0].get('external_metadata', {}).get('spotify', {}).get('track', {}).get('id')
print("Spotify Track ID:", spotify_track_id)


Spotify Track ID: 1pqT8dY3ti8vrjdxRnhi9g


## Spotify

In [10]:
SPOTIPY_CLIENT_ID = "997cffe9b6cc4226ab08f5ac9b12d4ed"
SPOTIPY_CLIENT_SECRET="ad98603976834fbeb322028814644db5"

spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(SPOTIPY_CLIENT_ID, SPOTIPY_CLIENT_SECRET))

In [11]:
track_info = spotify.track(spotify_track_id)

print(f"Track Name: {track_info['name']}")
print(f"Artist: {track_info['artists'][0]['name']}")
print(f"Album: {track_info['album']['name']}")
print(f"Release Date: {track_info['album']['release_date']}")
print(f"Preview URL: {track_info['preview_url']}")
print(f"Cover art: {track_info['album']['images'][0]['url']}")
print(f"URL to the song: {track_info['href']}")

Track Name: Hymn For The Weekend
Artist: Coldplay
Album: NOW That's What I Call Music!, Vol. 60 (Deluxe Edition)
Release Date: 2016-11-04
Preview URL: None
Cover art: https://i.scdn.co/image/ab67616d0000b273b31bf4e1f1f7d2937f8052b0
URL to the song: https://api.spotify.com/v1/tracks/1pqT8dY3ti8vrjdxRnhi9g


## Packaging all:

In [1]:
import base64
import hashlib
import hmac
import os
import time
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json

In [2]:
##### GLOBAL VARIABLES FOR CREDENTIALS:

# ACRCloud API credentials
ACCESS_KEY = ""
ACCESS_SECRET = ""
REQURL = "https://identify-eu-west-1.acrcloud.com/v1/identify" 

# Spotify API credentials
SPOTIPY_CLIENT_ID = ""
SPOTIPY_CLIENT_SECRET=""

# File path to the audio clip
AUDIO_FILE_PATH = 'DATASET/Test.mp4'

In [6]:
def get_similar_song_from_spotify(ACRCloud_access_key, ACRCloud_access_secret, ACRrequrl, Spotify_client_id, Spotify_client_secret, audio_file_path):
    # Request details
    http_method = "POST"
    http_uri = "/v1/identify"
    data_type = "audio"
    signature_version = "1"
    timestamp = time.time()

    # Create string to sign
    string_to_sign = "\n".join([
        http_method,
        http_uri,
        ACRCloud_access_key,
        data_type,
        signature_version,
        str(timestamp)
    ])

    # Generate signature
    sign = base64.b64encode(hmac.new(ACRCloud_access_secret.encode('ascii'), string_to_sign.encode('ascii'), hashlib.sha1).digest()).decode('ascii')

    # File to be recognized
    file_path = audio_file_path
    sample_bytes = os.path.getsize(file_path)

    # Prepare files and data for POST request
    files = {
        'sample': open(file_path, 'rb')
    }
    data = {
        'access_key': ACRCloud_access_key,
        'sample_bytes': sample_bytes,
        'timestamp': str(timestamp),
        'signature': sign,
        'data_type': data_type,
        'signature_version': signature_version
    }

    # Send request
    response = requests.post(ACRrequrl, files=files, data=data)
    response.encoding = "utf-8"
    recognition_result = response.text

    # Spotify ID
    result_json = json.loads(recognition_result)
    spotify_track_id = result_json.get('metadata', {}).get('music', [])[0].get('external_metadata', {}).get('spotify', {}).get('track', {}).get('id')

    # Request connection:
    spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(Spotify_client_id, Spotify_client_secret))

    # Get the song track
    track_info = spotify.track(spotify_track_id)


    # Return track_information
    return track_info

In [5]:

# Calling the function

track_info = get_similar_song_from_spotify(
    "a0915094ec173c83c245b22061073641",
    "uhlyu4xghQjwXB6qo2HDoi9uuLrDU9IYwK7RkOZt",
    "https://identify-eu-west-1.acrcloud.com/v1/identify",
    "997cffe9b6cc4226ab08f5ac9b12d4ed",
    "ad98603976834fbeb322028814644db5",
    'DATASET/Test.mp4'
)

Spotify Track ID: 1pqT8dY3ti8vrjdxRnhi9g


In [7]:
track_name = track_info['name']
artist_name = track_info['artists'][0]['name']
album_name = track_info['album']['name']
release_date = track_info['album']['release_date']
preview_url = track_info['preview_url']
cover_art = track_info['album']['images'][0]['url']
url = track_info['href']

print(f"Track Name: {track_name}")
print(f"Artist: {artist_name}")
print(f"Album: {album_name}")
print(f"Release Date: {release_date}")
print(f"Preview URL: {preview_url}")
print(f"Cover art: {cover_art}")
print(f"URL to the song: {url}")

Track Name: Hymn For The Weekend
Artist: Coldplay
Album: NOW That's What I Call Music!, Vol. 60 (Deluxe Edition)
Release Date: 2016-11-04
Preview URL: None
Cover art: https://i.scdn.co/image/ab67616d0000b273b31bf4e1f1f7d2937f8052b0
URL to the song: https://api.spotify.com/v1/tracks/1pqT8dY3ti8vrjdxRnhi9g
